Importing the libraries

In [23]:
import sklearn
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder
from sklearn import tree
from sklearn import preprocessing
import numpy as np

Read data file

In [24]:
df2 = pd.read_csv("out_without_nan_mood_target.csv")

Mood back to whole numbers

In [25]:
df=df2
df2['mood_mean_TARGET'] = df2['mood_mean_TARGET'].astype(float)
dfNonNan = df2.dropna(subset = ['mood_mean_TARGET'] )
dfNonNan = pd.concat([dfNonNan,pd.get_dummies(dfNonNan['id'])],axis=1)
df['mood_mean_TARGET'] = df['mood_mean_TARGET'].astype(float).round().astype(str)
df = pd.concat([df,pd.get_dummies(df['id'])],axis=1)

Id to dummy variable

Split in train and test set

In [26]:
corr=dfNonNan.corr()
goodColumns= corr[abs(corr['mood_mean_TARGET'])>0.25]['mood_mean_TARGET'].sort_values()
goodColumns = goodColumns.drop(['mood_mean_TARGET'])
print(goodColumns)

AS14.07                          -0.264961
circumplex.valence_mean_time_1    0.250137
circumplex.valence_mean           0.250137
circumplex.valence_mean_time_2    0.251291
mood_mean                         0.472047
mood_mean_time_1                  0.472047
mood_mean_time_3                  0.477074
mood_mean_time_4                  0.486367
mood_mean_time_5                  0.495198
mood_mean_time_2                  0.496638
Name: mood_mean_TARGET, dtype: float64


In [27]:
notRelevantAll=df.drop(['id','date'],axis=1).dropna()
goldY= notRelevantAll['mood_mean_TARGET']
relevant = notRelevantAll.drop(['mood_mean_TARGET'],axis=1)
X_train, X_test, y_train, y_test = train_test_split(relevant[goodColumns.keys()], goldY, test_size=0.33, random_state=42)

RANDOM FOREST CLASSIFIER

In [28]:
clf = RandomForestClassifier(n_estimators=10, max_depth=3,random_state=0)
clf = clf.fit(X_train, y_train)

In [29]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

scores = cross_val_score(clf, relevant[goodColumns.keys()], goldY, cv=7)
print(scores)
print(goldY.value_counts())
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=7.
  % (min_groups, self.n_splits)), Warning)


             precision    recall  f1-score   support

        3.0       0.00      0.00      0.00         1
        4.0       0.00      0.00      0.00         3
        5.0       0.00      0.00      0.00         7
        6.0       0.43      0.29      0.35        83
        7.0       0.63      0.78      0.70       226
        8.0       0.49      0.38      0.43        81
        9.0       0.00      0.00      0.00         1

avg / total       0.54      0.58      0.55       402

[0.53672316 0.48       0.56321839 0.52298851 0.61627907 0.63953488
 0.59302326]
7.0    613
8.0    301
6.0    259
5.0     22
9.0     11
4.0      8
3.0      2
Name: mood_mean_TARGET, dtype: int64
Accuracy: 0.56 (+/- 0.10)


GRADIENT BOOSTED CLASSIFIER

In [30]:
clf = GradientBoostingClassifier(n_estimators=20, max_depth=3,random_state=0,verbose=1)
clf = clf.fit(X_train, y_train)

      Iter       Train Loss   Remaining Time 
         1        1302.6177            0.25s
         2        1208.5844            0.23s
         3        1134.8368            0.23s
         4        1075.2139            0.21s
         5        1023.6881            0.20s
         6         979.6208            0.18s
         7         941.2040            0.17s
         8         906.5678            0.16s
         9         875.9362            0.15s
        10         849.9764            0.14s
        20         688.5247            0.00s


In [31]:
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

             precision    recall  f1-score   support

        3.0       0.00      0.00      0.00         1
        4.0       0.00      0.00      0.00         3
        5.0       0.00      0.00      0.00         7
        6.0       0.45      0.33      0.38        83
        7.0       0.65      0.78      0.71       226
        8.0       0.50      0.41      0.45        81
        9.0       0.00      0.00      0.00         1

avg / total       0.56      0.59      0.57       402



C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


## PER ID

In [32]:
for curID in df.id.unique():
    notRelevantAll=df[df['id']==curID]
    notRelevantAll=notRelevantAll.drop(['id','date'],axis=1).dropna()
    goldY= notRelevantAll['mood_mean_TARGET']
    relevant = notRelevantAll.drop(['mood_mean_TARGET'],axis=1)

    corr=dfNonNan[dfNonNan['id']==curID].corr()
    goodColumns= corr[abs(corr['mood_mean_TARGET'])>0.3]['mood_mean_TARGET'].sort_values()
    goodColumns = goodColumns.drop(['mood_mean_TARGET'])


    #X_train, X_test, y_train, y_test = train_test_split(relevant[goodColumns.keys()], goldY, test_size=0.25, random_state=42)
    clf = GradientBoostingClassifier(n_estimators=20, max_depth=3,random_state=0,verbose=0)
    #clf = clf.fit(X_train, y_train)
    #y_pred = clf.predict(X_test)
    scores = cross_val_score(clf, relevant[goodColumns.keys()], goldY, cv=7)
    print(curID)
    print(scores)
    print(goldY.value_counts())
    print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

AS14.01
[0.25       0.28571429 0.         0.28571429 0.66666667 0.2
 0.4       ]
7.0    19
8.0    15
6.0    11
Name: mood_mean_TARGET, dtype: int64
Accuracy: 0.30 (+/- 0.38)


C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=7.
  % (min_groups, self.n_splits)), Warning)


AS14.02
[0.4        0.33333333 0.66666667 0.66666667 1.         0.5
 0.        ]
7.0    18
6.0    15
8.0     4
4.0     1
5.0     1
9.0     1
Name: mood_mean_TARGET, dtype: int64
Accuracy: 0.51 (+/- 0.58)


C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=7.
  % (min_groups, self.n_splits)), Warning)


AS14.03
[0.375      0.71428571 0.42857143 0.57142857 1.         0.8
 0.6       ]
8.0    26
7.0    18
9.0     1
Name: mood_mean_TARGET, dtype: int64
Accuracy: 0.64 (+/- 0.40)


...

AS14.31
[0.57142857 0.85714286 0.71428571 0.8        1.         1.
 1.        ]
7.0    35
6.0     3
8.0     3
Name: mood_mean_TARGET, dtype: int64
Accuracy: 0.85 (+/- 0.31)


C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=7.
  % (min_groups, self.n_splits)), Warning)


AS14.32
[0.42857143 0.33333333 0.6        0.6        0.66666667 0.33333333
 0.33333333]
8.0    18
7.0    11
9.0     2
6.0     1
Name: mood_mean_TARGET, dtype: int64
Accuracy: 0.47 (+/- 0.27)


C:\Users\Rens\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:605: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=7.
  % (min_groups, self.n_splits)), Warning)


AS14.33
[0.57142857 0.42857143 0.28571429 0.5        0.6        0.75
 0.5       ]
6.0    17
7.0    14
8.0     5
5.0     4
Name: mood_mean_TARGET, dtype: int64
Accuracy: 0.52 (+/- 0.27)
